In [1]:
// Update the configuration of the Spark instance
// with the following AWS User Credentials
//// sc.hadoopConfiguration.set("fs.s3n.awsAccessKeyId", "XXX")
//// sc.hadoopConfiguration.set("fs.s3n.awsSecretAccessKey","XXX")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
23,application_1575849321448_0024,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
// Read GEOJSON of Chicago Business Permits Data, about 1.8GB
val df = spark.read.option("multiline",true).json( "s3n://201912bigdata210/chicagopermit.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [features: array<struct<geometry:struct<coordinates:array<double>,type:string>,properties:struct<application_start_date:string,building_fee_paid:string,building_fee_unpaid:string,building_fee_waived:string,census_tract:string,community_area:string,contact_10_city:string,contact_10_name:string,contact_10_state:string,contact_10_type:string,contact_10_zipcode:string,contact_11_city:string,contact_11_name:string,contact_11_state:string,contact_11_type:string,contact_11_zipcode:string,contact_12_city:string,contact_12_name:string,contact_12_state:string,contact_12_type:string,contact_12_zipcode:string,contact_13_city:string,contact_13_name:string,contact_13_state:string,... 94 more fields>,type:string>>, type: string]


In [3]:
df.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- features: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- geometry: struct (nullable = true)
 |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |-- properties: struct (nullable = true)
 |    |    |    |-- application_start_date: string (nullable = true)
 |    |    |    |-- building_fee_paid: string (nullable = true)
 |    |    |    |-- building_fee_unpaid: string (nullable = true)
 |    |    |    |-- building_fee_waived: string (nullable = true)
 |    |    |    |-- census_tract: string (nullable = true)
 |    |    |    |-- community_area: string (nullable = true)
 |    |    |    |-- contact_10_city: string (nullable = true)
 |    |    |    |-- contact_10_name: string (nullable = true)
 |    |    |    |-- contact_10_state: string (nullable = true)
 |    |    |    |-- contact_10_type: string (nullable = true)
 

In [4]:
// spark explode: Creates a new row for each element in the given array or map column. 
// help to flatten interesting parts of nested JSON
import org.apache.spark.sql.functions.explode

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions.explode


In [5]:
val dfGeometry = df.select(explode($"features.geometry")).select($"col.*").cache

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGeometry: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [coordinates: array<double>, type: string]


In [6]:
val cntGeo = dfGeometry.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cntGeo: Long = 606706


In [7]:
val dfProperties = df.select(explode($"features.properties")).select($"col.id", 
                                                              $"col.application_start_date",
                                                              $"col.community_area",
                                                              $"col.issue_date",
                                                              $"col.latitude",
                                                              $"col.longitude"
                                                             ).cache

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfProperties: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, application_start_date: string ... 4 more fields]


In [8]:
dfGeometry.count
dfProperties.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res10: Long = 606706
res11: Long = 606706


In [9]:
dfProperties.show(10, truncate = false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------------+--------------+-----------------------+------------+-------------+
|id     |application_start_date |community_area|issue_date             |latitude    |longitude    |
+-------+-----------------------+--------------+-----------------------+------------+-------------+
|2530006|2014-02-24T00:00:00.000|76            |2014-03-04T00:00:00.000|null        |null         |
|2506958|2013-11-18T00:00:00.000|76            |2013-12-19T00:00:00.000|null        |null         |
|2506971|2013-11-18T00:00:00.000|76            |2013-12-19T00:00:00.000|null        |null         |
|2818648|2017-01-25T00:00:00.000|28            |2017-01-25T00:00:00.000|null        |null         |
|1779434|2007-02-15T00:00:00.000|null          |2007-02-15T00:00:00.000|null        |null         |
|2616882|2014-12-04T00:00:00.000|7             |2014-12-20T00:00:00.000|null        |null         |
|3004830|2019-03-29T00:00:00.000|32            |2019-07-15T00:00:00.000|41.878542932|-87.632120577|


In [10]:
import org.apache.spark.sql.functions._

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._


In [11]:
val dfDateDif = dfProperties.withColumn("delta_date", datediff($"issue_date", $"application_start_date")
                                       ).withColumn("start_year", year(to_timestamp($"application_start_date"))
                                                   ).withColumn("zero_wait", when(col("delta_date") === 0, 1).otherwise(0))

dfDateDif.show(10, truncate = false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfDateDif: org.apache.spark.sql.DataFrame = [id: string, application_start_date: string ... 7 more fields]
+-------+-----------------------+--------------+-----------------------+------------+-------------+----------+----------+---------+
|id     |application_start_date |community_area|issue_date             |latitude    |longitude    |delta_date|start_year|zero_wait|
+-------+-----------------------+--------------+-----------------------+------------+-------------+----------+----------+---------+
|2530006|2014-02-24T00:00:00.000|76            |2014-03-04T00:00:00.000|null        |null         |8         |2014      |0        |
|2506958|2013-11-18T00:00:00.000|76            |2013-12-19T00:00:00.000|null        |null         |31        |2013      |0        |
|2506971|2013-11-18T00:00:00.000|76            |2013-12-19T00:00:00.000|null        |null         |31        |2013      |0        |
|2818648|2017-01-25T00:00:00.000|28            |2017-01-25T00:00:00.000|null        |null         |0 

In [14]:
val dfZeroWaitByYear = dfDateDif.groupBy($"start_year"
                                        ).agg(sum($"zero_wait") as "zero_wait"
                                              ,count($"zero_wait") as "total_permits"
                                             ).withColumn("zero_pct", round($"zero_wait" / $"total_permits",2)
                                                         ).sort($"start_year"
                                                               ).show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-------------+--------+
|start_year|zero_wait|total_permits|zero_pct|
+----------+---------+-------------+--------+
|      null|        0|          503|     0.0|
|      2001|        0|          588|     0.0|
|      2002|        0|          138|     0.0|
|      2003|        0|           77|     0.0|
|      2004|        0|          446|     0.0|
|      2005|        0|         2900|     0.0|
|      2006|    23762|        48251|    0.49|
|      2007|    21696|        44237|    0.49|
|      2008|    20633|        40765|    0.51|
|      2009|    22141|        39892|    0.56|
|      2010|    22657|        38830|    0.58|
|      2011|    21609|        36913|    0.59|
|      2012|    22448|        38681|    0.58|
|      2013|    23731|        41071|    0.58|
|      2014|    25922|        43461|     0.6|
|      2015|    29239|        44828|    0.65|
|      2016|    31178|        46044|    0.68|
|      2017|    31330|        47531|    0.66|
|      2018|    32281|        4680